In [10]:
from pathlib import Path

import numpy as np
import torch
import xarray as xr

from aurora import Batch, Metadata
import importlib

In [2]:
download_path = Path("/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/data")

day ='2022-02-01'


static_vars_ds = xr.open_dataset(download_path / "static.nc", engine="netcdf4")
surf_vars_ds = xr.open_dataset(download_path / day / f"{day}-surface-level.nc", engine="netcdf4")
atmos_vars_ds = xr.open_dataset(download_path / day / f"{day}-atmospheric.nc", engine="netcdf4")

atmos_vars_ds

<xarray.Dataset> Size: 1GB
Dimensions:              (time: 4, level: 13, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * level                (level) int32 52B 50 100 150 200 ... 700 850 925 1000
  * longitude            (longitude) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * time                 (time) datetime64[ns] 32B 2022-02-01 ... 2022-02-01T...
Data variables:
    temperature          (time, level, latitude, longitude) float32 216MB ...
    u_component_of_wind  (time, level, latitude, longitude) float32 216MB ...
    v_component_of_wind  (time, level, latitude, longitude) float32 216MB ...
    specific_humidity    (time, level, latitude, longitude) float32 216MB ...
    geopotential         (time, level, latitude, longitude) float32 216MB ...

In [3]:
x = [1, 2, 3, 4]
print(x.pop(0))
print(x)

1
[2, 3, 4]


In [4]:
for x in [1, 2, 3, 10, 12]:
    print(x, f'{x:02}')

1 01
2 02
3 03
10 10
12 12


In [21]:
from typing import Callable, List, Union, Tuple

class InferenceBatcher:
    def __init__(self, base_date_list: List[str], data_path: Path) -> None:
        self.base_date_list = base_date_list[:]
        self.day = self.base_date_list.pop(0)
        self.data_path = data_path
        self.static_vars_ds = xr.open_dataset(data_path / "static.nc", engine="netcdf4")
        self.surf_vars_ds: xr.Dataset
        self.atmos_vars_ds: xr.Dataset
        self._load_date_files()

        # Variable names
        self.surf_vars_names = [
            ('2t', '2m_temperature'),
            ('10u', '10m_u_component_of_wind'),
            ('10v', '10m_v_component_of_wind'),
            ('msl', 'mean_sea_level_pressure'),
        ]
        self.static_vars_names = [
            ('z', 'z'),
            ('slt', 'slt'),
            ('lsm', 'lsm')
        ]
        self.atmos_vars_names = [
            ('t', 'temperature'),
            ('u', 'u_component_of_wind'),
            ('v', 'v_component_of_wind'),
            ('q', 'specific_humidity'),
            ('z', 'geopotential')

        ]

        self.time_idx: int
        self.features: Batch
        self.labels: Batch
        self._set_initial_feature_labels()


    def _load_date_files(self) -> None:
        self.surf_vars_ds = xr.open_dataset(
            download_path / self.day / f"{self.day}-surface-level.nc",
            engine="netcdf4"
        )
        self.atmos_vars_ds = xr.open_dataset(
            download_path / self.day / f"{self.day}-atmospheric.nc",
            engine="netcdf4"
        )

    def _set_initial_feature_labels(self) -> None:
        self.time_idx = 0 # Initialized to 0
        self.features = self._make_batch()
        self.time_idx += 1
        self.labels = self._make_batch()
        self.time_idx += 1 # Finish at 2 --> this is the next index to pull from

    def _increment_day(self) -> None:
        # quick and dirty
        days_in_month = {2:28} # 2022 not a leap year
        for i in [9, 4, 6, 11]:
            days_in_month[i] = 30
        # all else is 31

        y, m, d = [int(x) for x in self.day.split('-')]
        d += 1
        if d > days_in_month.get(m, 31):
            m += 1
            d = 1
        assert m <= 12, f'Month is greater than 12: {m}'

        self.day = f'{y}-{m:02}-{d:02}'

    def _update_internal_state(self) -> bool:
        # First, check if time_index (i) is valid
        if self.time_idx > 3:
            # need to reload new date
            self._increment_day()

            # check whether the directory exists
            if (self.data_path / self.day).is_dir():
                # If next day directory exists, load from there
                self.time_idx = 0
                self._load_date_files()
            elif len(self.base_date_list):
                # If next day not found, need to jump to new base date
                self.day = self.base_date_list.pop(0)
                self._load_date_files()
                # Need to initialize new states for features and labels
                self._set_initial_feature_labels()
            else:
                return False

        return True
    
    def _update_features_and_labels(self) -> None:
        '''Updates internal state of features and labels'''
        # sh = short-hand, lh = long-hand
        self.features = Batch(
            surf_vars={
                sh:torch.concat((self.features.surf_vars[sh][:,[-1]], self.labels.surf_vars[sh][:, [-1]]), dim=1)
                for sh,_ in self.surf_vars_names
            },
            static_vars=self.labels.static_vars,
            atmos_vars={
                sh:torch.concat((self.features.atmos_vars[sh][:,[-1]], self.labels.atmos_vars[sh][:, [-1]]), dim=1)
                for sh,_ in self.atmos_vars_names
            },
            metadata=self.labels.metadata,
        )
        self.labels = self._make_batch()

    def _make_batch(self) -> Batch:
        def _prepare(x: np.ndarray) -> torch.Tensor:
            """Prepare a variable.

            This does the following things:
            * Select time indices `i` and `i - 1`.
            * Insert an empty batch dimension with `[None]`.
            * Flip along the latitude axis to ensure that the latitudes are decreasing.
            * Copy the data, because the data must be contiguous when converting to PyTorch.
            * Convert to PyTorch.
            """
            return torch.from_numpy(x[[self.time_idx]][None][..., ::-1, :].copy())

        return Batch(
            surf_vars={sh:_prepare(surf_vars_ds[lh].values) for sh,lh in self.surf_vars_names},
            static_vars={sh: torch.from_numpy(static_vars_ds[lh].values[0]) for sh,lh in self.static_vars_names},
            atmos_vars={sh: _prepare(atmos_vars_ds[lh].values) for sh,lh in self.atmos_vars_names},
            metadata=Metadata(
                # Flip the latitudes! We need to copy because converting to PyTorch, because the
                # data must be contiguous.
                lat=torch.from_numpy(surf_vars_ds.latitude.values[::-1].copy()),
                lon=torch.from_numpy(surf_vars_ds.longitude.values),
                # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
                # `datetime.datetime`s. Note that this needs to be a tuple of length one:
                # one value for every batch element.
                time=(surf_vars_ds.time.values.astype("datetime64[s]").tolist()[self.time_idx],),
                atmos_levels=tuple(int(level) for level in atmos_vars_ds.level.values),
            ),
        )

    def get_batch(self):
        is_valid_batch: bool = self._update_internal_state()

        if not is_valid_batch:
            return None, None
        else:
            self._update_features_and_labels()
            self.time_idx += 1
            return self.features, self.labels

In [27]:
batcher = InferenceBatcher(base_date_list=["2022-04-01", "2022-08-01"], data_path=download_path)

In [28]:
i = 0
while True:
    feats, labels = batcher.get_batch()
    if feats is None or labels is None:
        assert False, f'{i} --> NONE'
    print(i, feats.surf_vars['2t'].shape, labels.surf_vars['2t'].shape, batcher.time_idx - 1, batcher.day)
    i += 1

0 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 2 2022-04-01
1 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 3 2022-04-01
2 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 0 2022-04-02
3 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 1 2022-04-02
4 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 2 2022-04-02
5 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 3 2022-04-02
6 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 0 2022-04-03
7 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 1 2022-04-03
8 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 2 2022-04-03
9 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 3 2022-04-03
10 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 0 2022-04-04
11 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 1 2022-04-04
12 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 2 2022-04-04
13 torch.Size([1, 2, 7

AssertionError: 164 --> NONE

In [11]:
from aurora import inference_helper

def reload():
    importlib.reload(inference_helper)

In [13]:
reload()
batcher = inference_helper.InferenceBatcher(base_date_list=["2022-02-01", "2022-07-01"], data_path=download_path)

In [14]:
i = 0
while True:
    feats, labels = batcher.get_batch()
    if feats is None or labels is None:
        assert False, f'{i} --> NONE'
    print(i, feats.surf_vars['2t'].shape, labels.surf_vars['2t'].shape, batcher.time_idx - 1, batcher.day)
    i += 1
    break

0 torch.Size([1, 2, 721, 1440]) torch.Size([1, 1, 721, 1440]) 2 2022-02-01


In [23]:
from aurora import Aurora, rollout

model = Aurora()
# model.load_checkpoint("microsoft/aurora", "aurora-0.25-finetuned.ckpt")
model.load_checkpoint_local(
    "/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/models/hf_ckpt/aurora-0.25-finetuned.ckpt"
)

In [24]:
model.eval()
# model = torch.compile(model)

Aurora(
  (encoder): Perceiver3DEncoder(
    (surf_mlp): MLP(
      (net): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=512, bias=True)
        (3): Dropout(p=0.0, inplace=False)
      )
    )
    (surf_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (pos_embed): Linear(in_features=512, out_features=512, bias=True)
    (scale_embed): Linear(in_features=512, out_features=512, bias=True)
    (lead_time_embed): Linear(in_features=512, out_features=512, bias=True)
    (absolute_time_embed): Linear(in_features=512, out_features=512, bias=True)
    (atmos_levels_embed): Linear(in_features=512, out_features=512, bias=True)
    (surf_token_embeds): LevelPatchEmbed(
      (weights): ParameterDict(
          (10u): Parameter containing: [torch.FloatTensor of size 512x1x2x4x4]
          (10v): Parameter containing: [torch.FloatTensor of size 512x1x2x4x4]
    

In [20]:
model.eval()
# model = model.to("mps")

with torch.inference_mode():
    preds = [pred.to("cpu") for pred in rollout(model, feats, steps=1)]

# model = model.to("cpu")

KeyboardInterrupt: 

# Custom eval loop

In [21]:
import dataclasses
from datetime import timedelta

from aurora.model.fourier import lead_time_expansion

def custom_model_forward(model: Aurora, batch: Batch) -> Batch:
    """Forward pass.

    Args:
        batch (:class:`Batch`): Batch to run the model on.

    Returns:
        :class:`Batch`: Prediction for the batch.
    """
    # Get the first parameter. We'll derive the data type and device from this parameter.
    p = next(model.parameters())
    batch = batch.type(p.dtype)
    batch = batch.normalise(surf_stats=batch.surf_stats)
    batch = batch.crop(patch_size=model.patch_size)
    batch = batch.to('mps')

    H, W = batch.spatial_shape
    patch_res = (
        model.encoder.latent_levels,
        H // model.encoder.patch_size,
        W // model.encoder.patch_size,
    )

    # Insert batch and history dimension for static variables.
    B, T = next(iter(batch.surf_vars.values())).shape[:2]
    batch = dataclasses.replace(
        batch,
        static_vars={k: v[None, None].repeat(B, T, 1, 1) for k, v in batch.static_vars.items()},
    )

    encoder = model.encoder.to('mps')
    with torch.inference_mode():
        x = encoder(
            batch,
            lead_time=timedelta(hours=6),
        )
    encoder.to('cpu')

    # ---------------------------------------------------------------------------------------------
    # Backbone setup
    # ---------------------------------------------------------------------------------------------

    lead_time = timedelta(hours=6)
    rollout_step = batch.metadata.rollout_step

    # Yanked from swin3d forward
    _msg = "Input shape does not match patch size."
    assert x.shape[1] == patch_res[0] * patch_res[1] * patch_res[2], _msg

    # It's costly to pad across the level dimension, so we should not even though our model
    # supports it.
    _msg = f"Patch height ({patch_res[0]}) must be divisible by ws[0] ({model.backbone.window_size[0]})"
    assert patch_res[0] % model.backbone.window_size[0] == 0, _msg

    all_enc_res, padded_outs = model.backbone.get_encoder_specs(patch_res)
    all_enc_res

    lead_hours = lead_time / timedelta(hours=1)
    lead_times = lead_hours * torch.ones(x.shape[0], dtype=torch.float32, device=x.device)
    c = model.backbone.time_mlp(lead_time_expansion(lead_times, model.backbone.embed_dim).to(dtype=x.dtype))

    skips = []
    for i, layer in enumerate(model.backbone.encoder_layers):
        layer = layer.to('mps')
        with torch.inference_mode():
            x, x_unscaled = layer(x, c, all_enc_res[i], rollout_step=rollout_step)
        skips.append(x_unscaled)
        layer = layer.layer.to('cpu')

    for i, layer in enumerate(model.backbone.decoder_layers):
        index = model.backbone.num_decoder_layers - i - 1
        layer = layer.to('mps')
        with torch.inference_mode():
            x, _ = layer(
                x,
                c,
                all_enc_res[index],
                padded_outs[index - 1],
                rollout_step=rollout_step,
            )
        layer = layer.to('cpu')

        if 0 < i < model.backbone.num_decoder_layers - 1:
            # For the intermediate stages, we use additive skip connections.
            x = x + skips[index - 1]
        elif i == model.backbone.num_decoder_layers - 1:
            # For the last stage, we perform concatentation like in Pangu.
            x = torch.cat([x, skips[0]], dim=-1)
    # return x

    # ---------------------------------------------------------------------------------------------
    # END Backbone setup
    # ---------------------------------------------------------------------------------------------

    decoder = model.decoder.to('mps')
    with torch.inference_mode():
        pred = decoder(
            x,
            batch,
            lead_time=timedelta(hours=6),
            patch_res=patch_res,
        )
    decoder = decoder.to('cpu')

    # Remove batch and history dimension from static variables.
    pred = dataclasses.replace(
        pred,
        static_vars={k: v[0, 0] for k, v in batch.static_vars.items()},
    )

    # Insert history dimension in prediction. The time should already be right.
    pred = dataclasses.replace(
        pred,
        surf_vars={k: v[:, None] for k, v in pred.surf_vars.items()},
        atmos_vars={k: v[:, None] for k, v in pred.atmos_vars.items()},
    )

    pred = pred.unnormalise(surf_stats=model.surf_stats)

    return pred

In [22]:
from typing import Generator

def custom_rollout(model: Aurora, batch: Batch, steps: int) -> Generator[Batch, None, None]:
    """Perform a roll-out to make long-term predictions.

    Args:
        model (:class:`aurora.model.aurora.Aurora`): The model to roll out.
        batch (:class:`aurora.batch.Batch`): The batch to start the roll-out from.
        steps (int): The number of roll-out steps.

    Yields:
        :class:`aurora.batch.Batch`: The prediction after every step.
    """
    # We will need to concatenate data, so ensure that everything is already of the right form.
    # Use an arbitary parameter of the model to derive the data type and device.
    p = next(model.parameters())
    batch = batch.type(p.dtype)
    batch = batch.crop(model.patch_size)
    batch = batch.to('mps')

    for _ in range(steps):
        pred = custom_model_forward(model, batch)

        yield pred

        # Add the appropriate history so the model can be run on the prediction.
        batch = dataclasses.replace(
            pred,
            surf_vars={
                k: torch.cat([batch.surf_vars[k][:, 1:], v], dim=1)
                for k, v in pred.surf_vars.items()
            },
            atmos_vars={
                k: torch.cat([batch.atmos_vars[k][:, 1:], v], dim=1)
                for k, v in pred.atmos_vars.items()
            },
        )




In [25]:
preds = [pred.to("cpu") for pred in custom_rollout(model, feats, steps=1)]

RuntimeError: MPS backend out of memory (MPS allocated: 18.11 GB, other allocations: 672.00 KB, max allowed: 18.13 GB). Tried to allocate 102.83 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [18]:
torch.backends.mps.is_available()

True

In [14]:
i = 1  # Select this time index in the downloaded data.



def _prepare(x: np.ndarray) -> torch.Tensor:
        """Prepare a variable.

        This does the following things:
        * Select time indices `i` and `i - 1`.
        * Insert an empty batch dimension with `[None]`.
        * Flip along the latitude axis to ensure that the latitudes are decreasing.
        * Copy the data, because the data must be contiguous when converting to PyTorch.
        * Convert to PyTorch.
        """
        return torch.from_numpy(x[[i - 1, i]][None][..., ::-1, :].copy())


batch = Batch(
    surf_vars={
        "2t": _prepare(surf_vars_ds["2m_temperature"].values),
        "10u": _prepare(surf_vars_ds["10m_u_component_of_wind"].values),
        "10v": _prepare(surf_vars_ds["10m_v_component_of_wind"].values),
        "msl": _prepare(surf_vars_ds["mean_sea_level_pressure"].values),
    },
    static_vars={
        # The static variables are constant, so we just get them for the first time. They
        # don't need to be flipped along the latitude dimension, because they are from
        # ERA5.
        "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
    },
    atmos_vars={
        "t": _prepare(atmos_vars_ds["temperature"].values),
        "u": _prepare(atmos_vars_ds["u_component_of_wind"].values),
        "v": _prepare(atmos_vars_ds["v_component_of_wind"].values),
        "q": _prepare(atmos_vars_ds["specific_humidity"].values),
        "z": _prepare(atmos_vars_ds["geopotential"].values),
    },
    metadata=Metadata(
        # Flip the latitudes! We need to copy because converting to PyTorch, because the
        # data must be contiguous.
        lat=torch.from_numpy(surf_vars_ds.latitude.values[::-1].copy()),
        lon=torch.from_numpy(surf_vars_ds.longitude.values),
        # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        # one value for every batch element.
        # time=(surf_vars_ds.valid_time.values.astype("datetime64[s]").tolist()[i],),
        time=(surf_vars_ds.time.values.astype("datetime64[s]").tolist()[i],),
        atmos_levels=tuple(int(level) for level in atmos_vars_ds.level.values),
    ),
)

In [18]:
batch.surf_vars['msl'].shape

torch.Size([1, 2, 721, 1440])